# **output file with codon usage for the 10 trials**

In [1]:
import json
import pandas as pd
from datetime import datetime

In [2]:
cds_names = open("data/redo_multDif_exons_2021_02_07/cdsNames_list.txt").readlines()
for i,cds in enumerate(cds_names): 
    cds_names[i]= cds.rstrip("\n")

model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))

ntrial = 10 

# **wranlge into codon counts**

In [6]:
#creating the empty dictionary
codon_change_dict = {}
for key in model.keys(): 
    codon_change_dict[key] = []
    
codon_final_count_dict = {}
for key in model.keys(): 
    codon_final_count_dict[key] = []

for n in range(1, ntrial+1): 
    #make the edict that counts all triplets for each trial 
    curTrial_codon_change_dict  = {}
    for key in codon_change_dict.keys(): 
        curTrial_codon_change_dict[key] = {"initial":0, "6x":0}
    
    #start counting the triplets across all cds used 
    for cds_name in cds_names: 
        #read in the intiial dna 
        cur_dna_gen0_filepath= "data/redo_multDif_exons_2021_02_07/CodingUsage_cds{c}_DNA_gen0.txt".format(c=cds_name)
        cur_dna_gen0_str = open(cur_dna_gen0_filepath).readlines()[0]
        #read in the 6x coverage dna 
        cur_dna_filepath = "data/redo_multDif_exons_2021_02_07/CodingUsage_cds{c}_Trial{n}_DNA_6x.txt".format(c=cds_name, n=n)
        cur_dna_str = open(cur_dna_filepath).readlines()[0]
        
        #codon initial count
        for i in range(1, len(cur_dna_gen0_str)-1, 3): #verified to count codons in a strign mutliple of 3 
            cur_triplet = cur_dna_gen0_str[i-1: i+2]
            curTrial_codon_change_dict[cur_triplet]["initial"]+=1
        #count the 6x dna 
        for i in range(1, len(cur_dna_str)-1, 3): 
            cur_triplet = cur_dna_str[i-1: i+2]
            curTrial_codon_change_dict[cur_triplet]["6x"]+=1
            
    #calculate the prop chnage and append to the codonchnage dict
    for key,value in curTrial_codon_change_dict.items():
        cur_prop_chnage = (value["6x"]-value["initial"])/value["initial"]
        codon_change_dict[key].append(cur_prop_chnage)
        
        #calculate the prop chnage and append to the codonchnage dict
    for codon,value in curTrial_codon_change_dict.items():
        codon_final_count_dict[codon].append(value["6x"])

# **now convert into usage bias**

In [10]:
SynonymousCodons = {'CYS': ['TGT', 'TGC'], 'ASP': ['GAT', 'GAC'],
'SER': ['TCT', 'TCG', 'TCA', 'TCC', 'AGC', 'AGT'],
'GLN': ['CAA', 'CAG'], 'MET': ['ATG'], 'ASN': ['AAC', 'AAT'],
'PRO': ['CCT', 'CCG', 'CCA', 'CCC'], 'LYS': ['AAG', 'AAA'],
'STOP': ['TAG', 'TGA', 'TAA'], 'THR': ['ACC', 'ACA', 'ACG', 'ACT'],
'PHE': ['TTT', 'TTC'], 'ALA': ['GCA', 'GCC', 'GCG', 'GCT'],
'GLY': ['GGT', 'GGG', 'GGA', 'GGC'], 'ILE': ['ATC', 'ATA', 'ATT'],
'LEU': ['TTA', 'TTG', 'CTC', 'CTT', 'CTG', 'CTA'], 'HIS': ['CAT', 'CAC'],
'ARG': ['CGA', 'CGC', 'CGG', 'CGT', 'AGG', 'AGA'], 'TRP': ['TGG'],
'VAL': ['GTA', 'GTC', 'GTG', 'GTT'], 'GLU': ['GAG', 'GAA'], 'TYR': ['TAT', 'TAC']}

In [22]:
codon_usage_dict = {}
for codon in model.keys(): 
    codon_usage_dict[codon]=[]
    
for trial in range(10): 
    for aa,codon_list in SynonymousCodons.items(): 
        #getting the coutns for all synonymous codons 
        codon_count_list = []
        for codon in codon_list: 
            codon_count_list.append(codon_final_count_dict[codon][trial])
        #finding the codon with max count (to use for relative usage bias)
        index_max_count = codon_count_list.index(max(codon_count_list))
        codon_max_count = (codon_list[index_max_count])
        #getting the codon usage bias 
        for codon in codon_list: 
            codon_usage = int(codon_final_count_dict[codon][trial])/codon_final_count_dict[codon_max_count][trial]
            codon_usage_dict[codon].append(codon_usage)
            
    

In [25]:
#double checking --> verified using synonymous cocodn for CYS 
# print(codon_final_count_dict["TGT"])
# print(codon_final_count_dict["TGC"])
# print(codon_usage_dict["TGT"])
# print(codon_usage_dict["TGC"])

[539, 537, 520, 536, 527, 540, 524, 525, 533, 528]

In [29]:
codon_usage_df = pd.DataFrame.from_dict(codon_usage_dict)
(codon_usage_df.index) = ["Trial"+str(n) for n in range(1,ntrial+1)] #rename rows to trials

In [30]:
codon_usage_df

,TGC,GCG,CGG,AGA,GTT,TTG,GTG,ATA,TAA,TGA,...,ACG,GAG,CTG,CTA,GAT,GTC,GGT,TCG,TAT,AAA
Trial1,0.684601,0.525664,0.363107,1.0,1.000000,1.000000,0.963810,0.897547,1.0,1.0,...,0.352470,0.813846,0.841615,0.902174,1.0,0.832381,1.000000,0.433148,1.0,1.0
Trial2,0.679702,0.505119,0.385246,1.0,0.973832,0.998457,1.000000,0.993837,1.0,1.0,...,0.315327,0.823256,0.814815,0.844136,1.0,0.760748,0.993852,0.413462,1.0,1.0
Trial3,0.696154,0.512367,0.391667,1.0,1.000000,1.000000,0.977011,0.961595,1.0,1.0,...,0.330667,0.872204,0.815830,0.858447,1.0,0.831418,0.975359,0.408054,1.0,1.0
Trial4,0.701493,0.506151,0.376494,1.0,1.000000,1.000000,0.930841,0.938484,1.0,1.0,...,0.329857,0.842688,0.834891,0.890966,1.0,0.796262,1.000000,0.422709,1.0,1.0
Trial5,0.741935,0.487437,0.427632,1.0,1.000000,1.000000,0.986767,0.924088,1.0,1.0,...,0.304348,0.831761,0.804615,0.844615,1.0,0.826087,0.970060,0.410569,1.0,1.0
Trial6,0.729630,0.528881,0.389452,1.0,1.000000,1.000000,0.953618,1.000000,1.0,1.0,...,0.338255,0.841530,0.832013,0.946117,1.0,0.775510,1.000000,0.399736,1.0,1.0
Trial7,0.721374,0.506045,0.395876,1.0,1.000000,1.000000,0.907776,0.920896,1.0,1.0,...,0.327970,0.804970,0.813070,0.857143,1.0,0.770344,1.000000,0.401606,1.0,1.0
Trial8,0.666667,0.506993,0.361868,1.0,1.000000,1.000000,0.937063,0.938053,1.0,1.0,...,0.354520,0.868654,0.778452,0.878604,1.0,0.755245,1.000000,0.427577,1.0,1.0
Trial9,0.714822,0.506993,0.374000,1.0,1.000000,1.000000,0.985663,0.957317,1.0,1.0,...,0.370266,0.838407,0.824850,0.847305,1.0,0.745520,1.000000,0.405585,1.0,1.0
Trial10,0.719697,0.532628,0.372781,1.0,1.000000,1.000000,0.976492,0.906031,1.0,1.0,...,0.325581,0.869286,0.738872,0.848665,1.0,0.793852,1.000000,0.428369,1.0,1.0


In [32]:
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
codon_usage_df.to_csv("data/redo_multDif_exons_2021_02_07/{t}_codonUsage_multCds.csv".format(t=timestamp))